### Get basic data structures ready

In [1]:
rows = "ABCDEFGHI"
cols = "123456789"



def cross(a, b):
      return [s+t for s in a for t in b]

In [2]:
all_boxes = [r+c for r in rows for c in cols]

In [3]:
row_units = [[r + c for c in cols] for r in rows]

In [4]:
col_units = [[r + c for r in rows] for c in cols]

In [5]:
square_units = [cross (r, c) for r in ['ABC', 'DEF', 'GHI'] for c in ['123', '456', '789']]

In [6]:
unitlist = row_units + col_units + square_units

In [7]:
units = dict((s, [u for u in unitlist if s in u]) for s in all_boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in all_boxes)


## Some helper functions

In [8]:
# Convert string representation to key:value representation
def str_to_grid(string):
    values = dict((rows[r] + cols[c], string[r * 9 + c])  for c in range(9) for r in range(9))
    return values


def display_grid(grid):
    for r in rows:
        for c in cols:
            print('{0: <10}'.format(grid[r + c]), end='')
        print('')


def replace_dots_with_numbers(grid):
    for v in grid:
        if grid[v] == '.':
            grid[v] = '123456789'
            
    return grid

def validate(grid):
    for unit in unitlist:
        s = set()
        for u in unit:
            if grid[u] in list('123456789'):
                s.add(grid[u])
        if len(s) != 9:
            return False
        
    return True
            



## Elimination step

In this, we inspect boxes with one element. We know that that element will not repeat in any of the peers boxes. So we eliminate that element from all peer boxes.

In [9]:
def eliminate(grid):
    # extract all boxes with just one number
    singles = []
    for b in grid:
        if len(grid[b]) == 1:
            singles.append(b)
            
    for s in singles:
        for p in peers[s]:
            #print("Eliminating {0} from {1} at location {2}".format(grid[s], grid[p], p))
            grid[p] = grid[p].replace(grid[s], '')
    return grid
            

## Only Choice step

In this we look at boxes with multiple options, and inspect if there is one out of the multiple options which only that box can be assigned to. If so - we do the assignment.

In [10]:
def only_choice(grid):

    for unit in unitlist:
        for box in unit:
            v = grid[box]
            if len(v) > 1:
                for d in '123456789':
                    if d in v:
                        update = True
                        for box1 in unit:
                            if box != box1 and d in grid[box1]:
                                update = False
                        if update:
                            #print("Assigning {0} to {1}".format(d, grid[box]))
                            grid[box] = d
                    
    return grid
                        

## Constraing propagation

Apply Elimination and Only Choice steps in succession till the search space is reduced

In [11]:
def reduce_grid(grid):
    stagnent = False
    while not stagnent:
        #print('.')
        solved_values_before = len([box for box in grid.keys() if len(grid[box]) == 1])
        grid = eliminate(grid)
        grid = only_choice(grid)
        solved_values_after = len([box for box in grid.keys() if len(grid[box]) == 1])
        
        if solved_values_before == solved_values_after:
            stagnent = True
            
        if len([box for box in grid.keys() if len(grid[box]) == 0]):
            return False
    return grid

In [12]:
str_to_solve = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
str_to_solve = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
grid = str_to_grid(str_to_solve)
grid = replace_dots_with_numbers(grid)
display_grid(grid)
print('========')
grid = reduce_grid(grid)
print('========')
display_grid(grid)
validate(grid)


4         123456789 123456789 123456789 123456789 123456789 8         123456789 5         
123456789 3         123456789 123456789 123456789 123456789 123456789 123456789 123456789 
123456789 123456789 123456789 7         123456789 123456789 123456789 123456789 123456789 
123456789 2         123456789 123456789 123456789 123456789 123456789 6         123456789 
123456789 123456789 123456789 123456789 8         123456789 4         123456789 123456789 
123456789 123456789 123456789 123456789 1         123456789 123456789 123456789 123456789 
123456789 123456789 123456789 6         123456789 3         123456789 7         123456789 
5         123456789 123456789 2         123456789 123456789 123456789 123456789 123456789 
1         123456789 4         123456789 123456789 123456789 123456789 123456789 123456789 
4         1679      12679     139       2369      269       8         1239      5         
26789     3         1256789   14589     24569     245689    12679     1249      124679    

False

## Search


In [13]:
def search(grid):
    
    grid = reduce_grid(grid)
    
    if grid == False:
        return False
    
    index = None
    for b in grid:
        if len(grid[b]) > 1:
            index = b
            break
            
    if index is None:
        return grid
    
    elements = grid[index]
    
    for e in elements:
        grid_copy = grid.copy()
        grid_copy[index] = e
        attempt = search(grid_copy)
        if attempt:
            return attempt
        

        
        

In [14]:


def search1(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_grid(values)
    
    if values == False:
        return False
    
    # Choose one of the unfilled squares with the fewest possibilities
    
    v_index = None
    for v in values:
        if len(values[v]) > 1:
            v_index = v
            break
        
    if v_index is None:
        return values # Nothing to solve further
    
    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), return that answer!
    vlist = values[v_index]
    for d in vlist:
        vcopy = values.copy()
        vcopy[v_index] = d
        vattempt = search(vcopy)
        if vattempt:
            return vattempt
    

    # If you're stuck, see the solution.py tab!

In [14]:
str_to_solve = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
str_to_solve = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
grid = str_to_grid(str_to_solve)
grid = replace_dots_with_numbers(grid)
display_grid(grid)
grid = search(grid)
display_grid(grid)
validate(grid)


4         123456789 123456789 123456789 123456789 123456789 8         123456789 5         
123456789 3         123456789 123456789 123456789 123456789 123456789 123456789 123456789 
123456789 123456789 123456789 7         123456789 123456789 123456789 123456789 123456789 
123456789 2         123456789 123456789 123456789 123456789 123456789 6         123456789 
123456789 123456789 123456789 123456789 8         123456789 4         123456789 123456789 
123456789 123456789 123456789 123456789 1         123456789 123456789 123456789 123456789 
123456789 123456789 123456789 6         123456789 3         123456789 7         123456789 
5         123456789 123456789 2         123456789 123456789 123456789 123456789 123456789 
1         123456789 4         123456789 123456789 123456789 123456789 123456789 123456789 
4         1         7         3         6         9         8         2         5         
6         3         2         1         5         8         9         4         7         

True